<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/RedditConversationQualityCommentsExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparation

In [ ]:
!pip install praw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [ ]:
import praw
import json
from prawcore.exceptions import Forbidden
import time
import os
import unittest
import os
import time
import random
import signal
import time

In [ ]:
# Create a Reddit instance
reddit = praw.Reddit(client_id='4Fh9K7U29aRt6GUpOkoqfw', client_secret='tyuF_v56l2bFmSqUOXFldWPtYsFViA', user_agent='Simurgh__')

#Comment extraction

In [ ]:
def write_data_to_file(element, entry, id):
  with open(f"{base_path}{entry}s_{id}.json", "a") as f:
      json.dump(element, f)
      f.write("\n")

In [ ]:
# Function to write the progress to a file
def write_progress_to_file(last_processed_index, filename):
    with open(filename, 'w') as f:
        f.write(str(last_processed_index))

In [ ]:
import praw
import json
import time

def process_submission_comments(id):
    # Load or create the progress file
    progress_file = f'{base_path}comments_progress_{id}.txt'
    if os.path.isfile(progress_file):
        with open(progress_file, 'r') as f:
            last_processed_index = int(f.read())
            print("last_processed_index ", last_processed_index)
    else:
        last_processed_index = 0

    # Open the file containing the submission IDs and number of comments
    with open(f'{base_path}submissions_{id}.json', 'r') as f:
        submission_data = [json.loads(line) for line in f]

    # Iterate over the submission IDs and retrieve the comments for each submission
    for index, data in enumerate(submission_data[last_processed_index:], start=last_processed_index ):
        if last_processed_index != 0:
            index += 1
        #try:
        #   submission_data[index]

        #except IndexError:
        #   print("extraction is over for id.", id)
        #   break
        print("current submission index ", index)
        # Get the submission ID and number of comments
        submission_id = data['id']
        problematic_ids = ["xxjagq", "xyswda", "xysvkb", "xxjb63", "xsodkr", "xsobpb", "xuau8m", "xuf428", "xv1128", "xv100m", "xv0yjd", "xv0yiy", "xv0wu8", "xw5ezo", "xw5eo0"]#xuf6e4 xv1128
        if submission_id in problematic_ids:
          print("skipping id ",submission_id, " inaccessible for unknown reason ...")
          write_progress_to_file(index, progress_file)
          continue
        num_comments = data['num_comments']

        if num_comments == 0:
            print(f"Submission ID {submission_id} has zero comments, skipping...")
            write_progress_to_file(index, progress_file)
            continue
        if num_comments > 1000:
            print(f"Submission ID {submission_id},  {num_comments} comments, skipping for now...")
            write_progress_to_file(index, progress_file)
            continue

        # Get the submission object
        # print("id ", submission_id, " num of comments ", num_comments)
        try:
            # Get the submission object
            submission = reddit.submission(id=submission_id)
            print("submission ", submission.id)
            # Check if the submission is public and accessible
            if submission.author is not None:
                print(f"Submission ID {submission_id} is public and accessible. with {num_comments} comments.")
            else:
                print(f"Submission ID {submission_id} is either private or inaccessible, skipping...")
                write_progress_to_file(index, progress_file)
                continue

        except Forbidden:
            print("Access to the submission is forbidden, skipping...")
            write_progress_to_file(index, progress_file)
            continue

        while True:
            try:
                # Get all of the comments
                submission.comments.replace_more(limit=None)
                break
            except Forbidden:
                # If we get a 403 Forbidden error, wait for 10 seconds and try again
                print("Got a 403 error. Waiting for 30 seconds and trying again...")
                time.sleep(30)
        # Collect the comment data with specified keys
        comments = []
        comment_keys = ['all_awardings', 'approved_at_utc', 'approved_by', 'archived', 'associated_award', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'award', 'awarders', 'banned_at_utc', 'banned_by', 'block', 'body', 'body_html', 'can_gild', 'can_mod_post', 'clear_vote', 'collapse', 'collapsed', 'collapsed_because_crowd_control', 'collapsed_reason', 'collapsed_reason_code', 'comment_type', 'controversiality', 'created', 'created_utc', 'delete', 'depth', 'disable_inbox_replies', 'distinguished', 'downs', 'downvote', 'edit', 'edited', 'enable_inbox_replies', 'fullname', 'gild', 'gilded', 'gildings', 'id', 'id_from_url', 'is_root', 'is_submitter', 'likes', 'link_id', 'locked', 'mark_read', 'mark_unread', 'mod', 'mod_note', 'mod_reason_by', 'mod_reason_title', 'mod_reports', 'name', 'no_follow', 'num_reports', 'parent', 'parent_id', 'parse', 'permalink', 'refresh', 'removal_reason', 'replies', 'reply', 'report', 'report_reasons', 'save', 'saved', 'score', 'score_hidden', 'send_replies', 'stickied', 'submission', 'top_awarded_type', 'total_awards_received', 'treatment_tags', 'uncollapse', 'unrepliable_reason', 'unsave', 'ups', 'upvote', 'user_reports']

        for comment in submission.comments.list():
            comment_data = {}
            for key in comment_keys:
                if hasattr(comment, key):
                    obj = getattr(comment, key)
                    if isinstance(obj, dict) or isinstance(obj, list) or isinstance(obj, int) or isinstance(obj, float) or isinstance(obj, str) or isinstance(obj, bool):
                        comment_data[key] = getattr(comment, key)
            comments.append(comment_data)

        # Create the submission data
        submission_data = {
            'submission_id': submission_id,
            'num_comments': num_comments,
            'comments': comments
        }

        # Save the comment data to a JSON file
        write_data_to_file(submission_data, "comment", id)
        # Update the progress file with the current index
        write_progress_to_file(index, progress_file)

#Executor

In [ ]:
# Define the function that executes another function with a timeout
def execute_with_timeout(func, timeout_duration, *args, **kwargs):
    # Define the handler for the interrupt signal
    def handler(signum, frame):
        raise TimeoutError("Function execution timed out")

    # Set the signal handler
    signal.signal(signal.SIGALRM, handler)

    try:
        # Start the timer
        signal.alarm(timeout_duration)

        # Execute the function with the provided arguments
        func(*args, **kwargs)

    except TimeoutError:
        # Handle the timeout error
        print("Function execution timed out")

    finally:
        # Disable the alarm
        signal.alarm(0)

In [ ]:
def copy_submission_id(submission_progress_filename, comment_progress_file_name):
    with open(submission_progress_filename, 'r') as file:
        lines = file.readlines()

    with open(comment_progress_file_name, 'w') as new_file:
        for line in lines:
            values = line.strip().split(',')
            id = values[0].strip()
            new_line = f"{id},0\n"
            new_file.write(new_line)
            new_file.flush()

    print(f"New file '{comment_progress_file_name}' created with the copied IDs.")

In [ ]:
def update_comment_progress_file(file_path, id, status):
    print("file path", file_path, " id ", id, " status ", status)
    # Read the contents of the file
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Find the line with the starting timestamp
    for i, line in enumerate(lines):
        line_id, old_status = line.strip().split(',')
        if line_id == id:
            # Update the status to the new value
            lines[i] = f"{line_id},{status}\n"
            break

    # Write the updated contents back to the file
    with open(file_path, 'w') as f:
        f.writelines(lines)
        f.flush()

In [ ]:
def extract(submission_file_path, processor_func):
    file_path = f'{submission_file_path}_comments_progress.txt'
    if not os.path.exists(file_path):
        copy_submission_id(submission_file_path, file_path)

    iterated_ids = []
    while True:
        all_done = True
        with open(file_path, "r") as f:
            for line in f:
                id, done_str = line.strip().split(",")
                done = int(done_str) == 1
                if not done:
                    all_done = False
                    break
        if all_done:
          break
        processor_func(id)
        iterated_ids.append(id)
        update_comment_progress_file(file_path, id, 1)
    return iterated_ids

In [ ]:
import unittest
import os

# Define a test class
class ExtractTestCase(unittest.TestCase):
    # Set up any necessary test data before each test case
    def setUp(self):
        # Create a test submission file with 20 lines
        self.submission_file_path = 'test_submission_file.txt'
        self.progress_file_path = f'{self.submission_file_path}_comments_progress.txt'
        with open(self.submission_file_path, 'w') as file:
            for i in range(20):
                file.write(f"{i+1},{i+1},0\n")
        if os.path.exists(self.progress_file_path):
            os.remove(self.progress_file_path)

    # Clean up any resources after each test case
    def tearDown(self):
        # Remove the test submission file
        if os.path.exists(self.submission_file_path):
            os.remove(self.submission_file_path)
        if os.path.exists(self.progress_file_path):
            os.remove(self.progress_file_path)

    # Test scenario 1: Progress file doesn't exist at the start
    def test_progress_file_creation(self):
        print("test_progress_file_creation")
        # Ensure the progress file doesn't exist initially
        self.assertFalse(os.path.exists(self.progress_file_path))

        # Call the extract function
        result = extract(self.submission_file_path, lambda x: None)

        # Ensure the progress file is created
        self.assertTrue(os.path.exists(self.progress_file_path))

        # Ensure the extract function iterated through all IDs (lines)
        self.assertEqual(len(result), 20)

    # Test scenario 2: Set progress status to 1 for some lines
    def test_set_progress_status(self):
        print("test_set_progress_status")
        # Create the progress file with some lines set to 1
        copy_submission_id(self.submission_file_path, self.progress_file_path)
        update_comment_progress_file(self.progress_file_path, "5", 1)
        update_comment_progress_file(self.progress_file_path, "10", 1)

        # Call the extract function
        result = extract(self.submission_file_path, lambda x: None)

        # Ensure the extract function iterated from line 6 to line 20
        self.assertEqual(len(result), 18)

    # Test scenario 3: Interrupt function execution and check progress file changes
    def test_interrupt_function_execution(self):
        print("test_interrupt_function_execution")
        # Define a mock function to be passed as an argument
        def mock_func(data_id):
            print(f"Interupted, Processing data ID: {data_id}")
            # Simulate a long execution
            time.sleep(1)

        # Call the function being tested
        execute_with_timeout(extract, 5, self.submission_file_path, mock_func)

        # Check the progress file for changes
        with open(self.progress_file_path, 'r') as file:
            lines = file.readlines()

        # Ensure the progress status of all lines is 1
        processed = 0
        for line in lines:
           id, status = line.strip().split(',')
           if status == "1":
              processed += 1
        # Call the extract function with the mock processor function
        output = extract(self.submission_file_path, mock_func)
        self.assertEqual(processed + len(output), 20)


#Extraction call

In [ ]:
from prawcore.exceptions import Forbidden, TooManyRequests
while True:
  try:
    extract(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt", process_submission_comments)
  except TooManyRequests:
    # If we get a 429 TooManyRequests error, wait for 60 seconds and try again
    print("Got a 429 error (Too Many Requests). Waiting for 60 seconds and trying again...")
    time.sleep(60)

#Test execution call

In [ ]:
test_case = ExtractTestCase()
unittest.main(argv=[''], exit=False)

#Comments key processing

In [ ]:
import praw
import openpyxl
import os.path

# Check if the comment keys file exists
if os.path.exists(f'{base_path}comment_keys.xlsx'):
    print("Comment keys file already exists. Skipping rewriting.")
else:
    # Create a Reddit instance
    reddit = praw.Reddit(client_id='4Fh9K7U29aRt6GUpOkoqfw', client_secret='tyuF_v56l2bFmSqUOXFldWPtYsFViA', user_agent='Simurgh__')

    # Get a submission
    submission_id = 'xz45qo'
    submission = reddit.submission(id=submission_id)

    # Get a comment
    comment = submission.comments.list()[0]  # Get the first comment


    # Load submission fields from submission_keys.xlsx
    submission_workbook = openpyxl.load_workbook(f'{base_path}submission_keys.xlsx')
    submission_sheet = submission_workbook.active

    # Create a new Excel workbook
    comment_workbook = openpyxl.Workbook()
    comment_sheet = comment_workbook.active

    # Set the first row as header
    comment_sheet.cell(row=1, column=1).value = "field"

    # Write the comment keys to the first column
    fields = dir(comment)
    row_index = 2
    for field in fields:
        if not field.startswith('_'):
            comment_sheet.cell(row=row_index, column=1).value = field
            comment_sheet.cell(row=row_index, column=2).value = field in [cell.value for cell in submission_sheet['A']]
            row_index += 1

    # Save the workbook as an Excel file
    comment_workbook.save(f'{base_path}comment_keys.xlsx')

In [ ]:
import openpyxl

# Load the comment keys Excel file
workbook = openpyxl.load_workbook(f'{base_path}comment_keys.xlsx')
sheet = workbook.active

# Initialize an empty array to store the fields
fields = []

# Iterate over the rows starting from the second row
for row in sheet.iter_rows(min_row=2, values_only=True):
    comment_field = row[0]
    exists_in_submission = row[2]

    # Check if the value in the third column is FALSE
    if exists_in_submission is False:
        fields.append(comment_field)

# Print the fields in an array-like string
print("Fields with value FALSE in the third column:")
print(str(fields))
print(len(fields))